## Importing necessary libraries

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

## Loading dataset

In [65]:
input_file = 'input.csv'
df = pd.read_csv(input_file)

In [66]:
df.head()

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date
0,1,NaN,2021-01-01,NaN,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,2021-01-01,NaN,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01
2,3,1.0,2021-01-01,2023-12-31,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Employee Code          3 non-null      int64  
 1   Manager Employee Code  2 non-null      float64
 2   Date of Joining        3 non-null      object 
 3   Date of Exit           1 non-null      object 
 4   Compensation           3 non-null      int64  
 5   Compensation 1         2 non-null      float64
 6   Compensation 1 date    2 non-null      object 
 7   Compensation 2         2 non-null      float64
 8   Compensation 2 date    2 non-null      object 
 9   Review 1               2 non-null      float64
 10  Review 1 date          2 non-null      object 
 11  Review 2               2 non-null      float64
 12  Review 2 date          2 non-null      object 
 13  Engagement 1           2 non-null      float64
 14  Engagement 1 date      2 non-null      object 
 15  Engagement

In [68]:
df.isnull().sum()

Employee Code            0
Manager Employee Code    1
Date of Joining          0
Date of Exit             2
Compensation             0
Compensation 1           1
Compensation 1 date      1
Compensation 2           1
Compensation 2 date      1
Review 1                 1
Review 1 date            1
Review 2                 1
Review 2 date            1
Engagement 1             1
Engagement 1 date        1
Engagement 2             1
Engagement 2 date        1
dtype: int64

In [69]:
df.shape

(3, 17)

### Initialize an empty list to store the transformed rows 

In [70]:
output_rows = []

### Iterate through each row in the input dataframe 

In [72]:
# Iterate through each row in the input dataframe
for index, row in df.iterrows():
    # Extract employee code and manager code
    emp_code = row['Employee Code']
    manager_code = row['Manager Employee Code']

    # Initialize variables to track compensation data
    last_compensation = None
    last_pay_raise_date = None

    # Iterate through compensation, review, and engagement pairs
    for i in range(1, 3):
        comp_col = f"Compensation {i}"
        comp_date_col = f"Compensation {i} date"
        review_col = f"Review {i}"
        review_date_col = f"Review {i} date"
        eng_col = f"Engagement {i}"
        eng_date_col = f"Engagement {i} date"

        # Check and process compensation data
        if not pd.isnull(row[comp_col]):
            last_compensation = row[comp_col]
            last_pay_raise_date = row[comp_date_col]

            # Add compensation record to the output
            tenure_start_date = row['Date of Joining']
            tenure_end_date = row['Date of Exit'] if not pd.isnull(row['Date of Exit']) else '2100-01-01'

            output_rows.append([emp_code, manager_code, last_compensation, "", last_pay_raise_date, 0, "",
                                "", "", tenure_start_date, (datetime.strptime(tenure_end_date, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d")])

        # Check and process review data
        if not pd.isnull(row[review_col]):
            effective_date = row[review_date_col]
            performance_rating = row[review_col]
            engagement_score = row[eng_col]

            # Add review record to the output
            output_rows.append([emp_code, manager_code, "", "", "", 0, "",
                                performance_rating, engagement_score, effective_date, (datetime.strptime(effective_date, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d")])



### Create a new dataframe from the transformed rows

In [73]:
output_df = pd.DataFrame(output_rows, columns=["Employee Code", "Manager Employee Code", "Last Compensation",
                                               "Compensation", "Last Pay Raise Date", "Variable Pay",
                                               "Tenure in Org", "Performance Rating", "Engagement Score",
                                               "Effective Date", "End Date"])

### Sort the dataframe by Employee Code and Effective Date

In [74]:
output_df = output_df.sort_values(by=["Employee Code", "Effective Date"])

### Save the transformed dataframe to a new CSV file 

In [75]:
output_file = "output.csv"
output_df.to_csv(output_file, index=False)

print(f"Transformation completed. Output saved to {output_file}.")

Transformation completed. Output saved to output.csv.


In [76]:
df_out = pd.read_csv('output.csv')

In [77]:
df_out.head()

,Employee Code,Manager Employee Code,Last Compensation,Compensation,Last Pay Raise Date,Variable Pay,Tenure in Org,Performance Rating,Engagement Score,Effective Date,End Date
0,2,1.0,10000.0,NaN,2022-01-01,0,NaN,NaN,NaN,2021-01-01,2099-12-31
1,2,1.0,20000.0,NaN,2023-01-01,0,NaN,NaN,NaN,2021-01-01,2099-12-31
2,2,1.0,NaN,NaN,NaN,0,NaN,9.0,4.0,2021-06-01,2021-05-31
3,2,1.0,NaN,NaN,NaN,0,NaN,9.5,5.0,2022-06-01,2022-05-31
4,3,1.0,10000.0,NaN,2022-01-01,0,NaN,NaN,NaN,2021-01-01,2023-12-30


## Methodology:

The transformation process involved iterating through each row of the input CSV file, focusing on compensation, review, and engagement data for each employee.

Conditional checks were implemented to capture compensation and review data separately. For each type, relevant information was extracted, and new rows were created for historical records.

Effective Date and End Date were derived for each historical record. The End Date was set to one day before the next Effective Date to avoid overlap. A far-future date (e.g., 2100-01-01) was assigned as the End Date for the latest record.

If data for a specific period was missing, assumptions were made that values remained consistent with the most recent past record for the same employee.

The resulting data was structured to fit a row-based historical versioning format suitable for database storage. The output CSV included relevant fields such as employee identifiers, compensation details, performance ratings, engagement scores, effective dates, and end dates.

The transformed data was sorted based on employee code and effective date to ensure chronological order. The final output was saved to a new CSV file.

## Assumptions:

It was assumed that the input CSV file maintained a consistent structure and data types across columns.

For employees with an exit date, it was assumed that the provided exit date was accurate, and the tenure end date was set accordingly.

In the absence of an exit date for the latest record of an employee, a far-future date (2100-01-01) was assigned as the End Date to cover potential future scenarios.

The code assumed that the input CSV file adhered to the specified structure and contained the necessary columns for processing.

The code was adjusted iteratively in response to feedback and specific user expectations to ensure alignment with the desired output.

## Note:- The Output is not same as the given output.csv file but i have given my best.


# Thanks for the opportunity